In [1]:
# load global and regional structural data
import pandas as pd

# my data file path
data = pd.read_csv('/data/sliu/updated_ukbb2/raw_data/matched_ukbb.csv')

# the items of global structural measures
total_items = pd.read_csv('/data/sliu/updated_ukbb2/raw_data/total_items.csv')
s1 = ['karin_IDs']
for i in range(total_items.shape[0]):
    s1.append(str(total_items.iloc[i,0])+'-2.0')

# the items of regional structural measures
regional_items = pd.read_csv('/data/sliu/updated_ukbb2/raw_data/regional_items_no_volumes.csv')
s2 = ['karin_IDs']
for i in range(regional_items.shape[0]):
    s2.append(str(regional_items.iloc[i,0])+'-2.0')

In [2]:
# total brain measures (total CSA, average CT and ICV)
T_data = data[s1]
T_data.dropna(axis=0,how='any',inplace=True)

# regional brain measures (CSA, CT based on DKT atlas)
R_data = data[s2]
R_data.dropna(axis=0,how='any',inplace=True)

/data/sliu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/data/sliu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [3]:
# load covariates (age, sex, head positions, the top 25 principal components)
temp = pd.read_csv('/data/sliu/updated_ukbb/added_used_variables.csv')
s = ['eid','25756-2.0','25757-2.0','25758-2.0']
QC1 = temp[s]
temp2 = pd.read_csv('/data/sliu/muti_PRSs/ukb30545.sample_QC.csv')
s2 = ['eid','age_at_reqruitment','genetic_sex']
QC2 = temp2[s2]
PCs = pd.read_table('/data/sliu/muti_PRSs/UKB.HM3.EUR.100PCs.txt','\t')

# load Polygenic scores for 14 mental health and cognitive traits
PRS_data = pd.read_csv('/data/sliu/updated_ukbb/ukb_PRSs.csv')

/data/sliu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: read_table is deprecated, use read_csv instead.
  


In [4]:
# matching subject IDs
T_data.set_index('karin_IDs', inplace=True)
R_data.set_index('karin_IDs', inplace=True)
QC1.set_index('eid',inplace=True)
QC2.set_index('eid',inplace=True)
PCs.set_index('IID',inplace=True)
PRS_data.set_index('FID',inplace=True)
l = list(set(T_data.index) & set(R_data.index) & set(QC1.index) & set(QC2.index) & set(PCs.index) & set(PRS_data.index))
final_Ts = T_data.loc[l]
final_Rs = R_data.loc[l]
final_QC1 = QC1.loc[l]
final_QC2 = QC2.loc[l]
final_PCs = PCs.loc[l]
final_PRSs = PRS_data.loc[l]

final_Ts.reset_index(inplace=True)
final_Rs.reset_index(inplace=True)
final_QC1.reset_index(inplace=True)
final_QC2.reset_index(inplace=True)
final_PCs.reset_index(inplace=True)
final_PRSs.reset_index(inplace=True)

In [6]:
# merge covariates together 
import numpy as np
PCs_25 = final_PCs.iloc[:,2:27].values
age = final_QC2.iloc[:,1:2].values
sex = final_QC2.iloc[:,2:3].values
postions = final_QC1.iloc[:,1:4].values
TA = (final_Ts.iloc[:,1:2].values + final_Ts.iloc[:,2:3].values)/2
AT = (final_Ts.iloc[:,3:4].values + final_Ts.iloc[:,4:5].values)/2
ICV = final_Ts.iloc[:,5:6].values
sex = sex + 1
co1 = np.hstack((PCs_25,age,sex,age*age,age*sex,age*age*sex,postions,TA)) # for CSA analysis
co2 = np.hstack((PCs_25,age,sex,age*age,age*sex,age*age*sex,postions,AT)) # for CT analysis
co3 = np.hstack((PCs_25,age,sex,age*age,age*sex,age*age*sex,postions,ICV)) # for analysis of subcortical volumes

In [7]:
# prepare structural data
area_items = pd.read_csv('/data/sliu/updated_ukbb2/raw_data/area_items.csv')
thickness_items = pd.read_csv('/data/sliu/updated_ukbb2/raw_data/thickness_items.csv')
volume_items = pd.read_csv('/data/sliu/updated_ukbb2/raw_data/sub_items.csv')
area_s = []
for i in range(area_items.shape[0]):
    area_s.append(str(area_items.iloc[i,0])+'-2.0')
thickness_s = []
for i in range(thickness_items.shape[0]):
    thickness_s.append(str(thickness_items.iloc[i,0])+'-2.0')
volume_s = []
for i in range(volume_items.shape[0]):
    volume_s.append(str(volume_items.iloc[i,0])+'-2.0')

In [11]:
SA_data = final_Rs[area_s].values
CT_data = final_Rs[thickness_s].values
volume_data = final_Rs[volume_s].values

In [12]:
# function used for regressing out the effects of covariates 
from sklearn.preprocessing import StandardScaler

def regression_covariant(covariant_matrix, y, standard_scale=False):
    a = np.hstack((covariant_matrix,np.ones((covariant_matrix.shape[0], 1))))
    w = np.linalg.lstsq(a,y,rcond=None)[0]

    residual = y - covariant_matrix.dot(w[:-1])
    residual = residual.astype('float64')

    if standard_scale:
        residual = StandardScaler().fit_transform(residual.reshape(-1,1)).flatten()

    return residual, w

In [13]:
# Pearson's correlation analysis between PRSs and CSA
from scipy.stats import pearsonr
X = final_PRSs.iloc[:,1:].values
X[:,7] = -X[:,7]
X[:,8] = -X[:,8]
X[:,11] = -X[:,11]
Y = SA_data
SA_R = np.empty((X.shape[1],Y.shape[1]))
SA_P = np.empty((X.shape[1],Y.shape[1]))
for i in range(X.shape[1]):
    for j in range(Y.shape[1]):
        x = X[:,i]
        y = Y[:,j]
        [rx,w1] = regression_covariant(co1,x,standard_scale=True)
        [ry,w1] = regression_covariant(co1,y,standard_scale=True)
        r,p = pearsonr(rx, ry)
        SA_R[i,j] = r
        SA_P[i,j] = p

In [14]:
# Pearson's correlation analysis between PRSs and CT
Y = CT_data
CT_R = np.empty((X.shape[1],Y.shape[1]))
CT_P = np.empty((X.shape[1],Y.shape[1]))
for i in range(X.shape[1]):
    for j in range(Y.shape[1]):
        x = X[:,i]
        y = Y[:,j]
        [rx,w1] = regression_covariant(co2,x,standard_scale=True)
        [ry,w1] = regression_covariant(co2,y,standard_scale=True)
        r,p = pearsonr(rx, ry)
        CT_R[i,j] = r
        CT_P[i,j] = p

In [15]:
# Pearson's correlation analysis between PRSs and subcortical volumes
Y = volume_data
volume_R = np.empty((X.shape[1],Y.shape[1]))
volume_P = np.empty((X.shape[1],Y.shape[1]))
for i in range(X.shape[1]):
    for j in range(Y.shape[1]):
        x = X[:,i]
        y = Y[:,j]
        [rx,w1] = regression_covariant(co3,x,standard_scale=True)
        [ry,w1] = regression_covariant(co3,y,standard_scale=True)
        r,p = pearsonr(rx, ry)
        volume_R[i,j] = r
        volume_P[i,j] = p

In [16]:
# merge their results (P values) together
total_P = np.hstack((SA_P,CT_P,volume_P))

In [17]:
# FDR multiple comparison correction
from statsmodels.stats import multitest
size = total_P.shape
temp_p = total_P.flatten()
Ps = multitest.multipletests(temp_p,alpha=0.05,method='fdr_bh')
P_corrected = Ps[1].reshape(size)

In [18]:
region_num = 33

In [19]:
SA_P_corrected = P_corrected[:,:2*region_num]
CT_P_corrected = P_corrected[:,2*region_num:4*region_num]
volume_P_corrected = P_corrected[:,4*region_num:]

In [20]:
# output CSA results
lSA_R = SA_R[:,:region_num]
lSA_P = SA_P[:,:region_num]
lSA_P_corrected = SA_P_corrected[:,:region_num]
re_l_R = pd.DataFrame(data=lSA_R)
re_l_R.to_csv('/data/sliu/updated_ukbb2/regional_results2/area_left_R.csv',header=None,index=False)
re_l_P = pd.DataFrame(data=lSA_P)
re_l_P.to_csv('/data/sliu/updated_ukbb2/regional_results2/area_left_P.csv',header=None,index=False)
re_l_P_corrected = pd.DataFrame(data=lSA_P_corrected)
re_l_P_corrected.to_csv('/data/sliu/updated_ukbb2/regional_results2/area_left_P_corrected.csv',header=None,index=False)

rSA_R = SA_R[:,region_num:]
rSA_P = SA_P[:,region_num:]
rSA_P_corrected = SA_P_corrected[:,region_num:]
re_r_R = pd.DataFrame(data=rSA_R)
re_r_R.to_csv('/data/sliu/updated_ukbb2/regional_results2/area_right_R.csv',header=None,index=False)
re_r_P = pd.DataFrame(data=rSA_P)
re_r_P.to_csv('/data/sliu/updated_ukbb2/regional_results2/area_right_P.csv',header=None,index=False)
re_r_P_corrected = pd.DataFrame(data=rSA_P_corrected)
re_r_P_corrected.to_csv('/data/sliu/updated_ukbb2/regional_results2/area_right_P_corrected.csv',header=None,index=False)

In [21]:
# output CT results
lCT_R = CT_R[:,:region_num]
lCT_P = CT_P[:,:region_num]
lCT_P_corrected = CT_P_corrected[:,:region_num]
re_l_R = pd.DataFrame(data=lCT_R)
re_l_R.to_csv('/data/sliu/updated_ukbb2/regional_results2/thickness_left_R.csv',header=None,index=False)
re_l_P = pd.DataFrame(data=lCT_P)
re_l_P.to_csv('/data/sliu/updated_ukbb2/regional_results2/thickness_left_P.csv',header=None,index=False)
re_l_P_corrected = pd.DataFrame(data=lCT_P_corrected)
re_l_P_corrected.to_csv('/data/sliu/updated_ukbb2/regional_results2/thickness_left_P_corrected.csv',\
                        header=None,index=False)

rCT_R = CT_R[:,region_num:]
rCT_P = CT_P[:,region_num:]
rCT_P_corrected = CT_P_corrected[:,region_num:]
re_r_R = pd.DataFrame(data=rCT_R)
re_r_R.to_csv('/data/sliu/updated_ukbb2/regional_results2/thickness_right_R.csv',header=None,index=False)
re_r_P = pd.DataFrame(data=rCT_P)
re_r_P.to_csv('/data/sliu/updated_ukbb2/regional_results2/thickness_right_P.csv',header=None,index=False)
re_r_P_corrected = pd.DataFrame(data=rCT_P_corrected)
re_r_P_corrected.to_csv('/data/sliu/updated_ukbb2/regional_results2/thickness_right_P_corrected.csv',\
                        header=None,index=False)

In [22]:
# output results of subcortical volumes 
subvol_R = volume_R
subvol_P = volume_P
subvol_P_corrected = volume_P_corrected
re_sub_R = pd.DataFrame(data=subvol_R)
re_sub_R.to_csv('/data/sliu/updated_ukbb2/regional_results2/volume_sub_R.csv',header=None,index=False)
re_sub_P = pd.DataFrame(data=subvol_P)
re_sub_P.to_csv('/data/sliu/updated_ukbb2/regional_results2/volume_sub_P.csv',header=None,index=False)
re_sub_P_corrected = pd.DataFrame(data=subvol_P_corrected)
re_sub_P_corrected.to_csv('/data/sliu/updated_ukbb2/regional_results2/volume_sub_P_corrected.csv',header=None,index=False)